In [46]:
import pandas as pd
import os
import logging
import numpy as np
from sklearn.metrics import classification_report
os.chdir('/Users/caicaiqijun/智慧牙/数据/TFFI/全国数据/模型/评级趋势优化')

# level

In [58]:
level=pd.read_excel('level.xlsx')
level

,rating_v2,level9
0,AAA,1
1,AA,2
2,A,3
3,BBB,4
4,BB,5
5,B,6
6,CCC,7
7,CC,8
8,C,9


# 测试集

In [23]:
def grade_fico(data,grade):
    original=data.copy()
    fico = original.loc[:,['company_id','score']]
    # fico['rating'] = fico.score*5.5+300
 
    # grade_fico = grade.at['fico_split_point','dim_level']
    # grade_fico=[0.547162092,4.999257361602687,20.4460314446204,33.704506754752465,45.460511959125675,56.5829515099279,66.67452317476864,75.62709131210073]
    grade_fico=grade
    print(grade_fico)
 
    fico['rating_v2'] = \
        np.where((fico.score.ge(grade_fico[7])
                    &original.value_sum.ge(2000000)
                    &original.granted_invention_cnt_pb_original_2y.ge(7)
                    &original.apno_cnt.ge(160)), 'AAA',
        np.where((fico.score.ge(grade_fico[7])), 'AA',
        np.where((fico.score.ge(grade_fico[6])
                    &original.value_sum.ge(650000)
                    &original.granted_invention_cnt_pb_original_2y.ge(2)
                    &original.apno_cnt.ge(70)),'AA',
        np.where((fico.score.ge(grade_fico[6])), 'A',
        np.where((fico.score.ge(grade_fico[5])
                    &original.value_sum.ge(250000)),'A',
        np.where((fico.score.ge(grade_fico[5])), 'BBB',
        np.where((fico.score.ge(grade_fico[4])
                    &original.value_sum.ge(250000)),'BBB',
        np.where((fico.score.ge(grade_fico[4])), 'BB',
        np.where((fico.score.ge(grade_fico[3])
                    &original.value_sum.ge(100000)),'BB',
        np.where((fico.score.ge(grade_fico[3])), 'B',
        np.where((fico.score.ge(grade_fico[2])
                    &original.value_sum.ge(40000)),'B',
        np.where((fico.score.ge(grade_fico[2])), 'CCC',
        np.where((fico.score.ge(grade_fico[1])), 'CCC',
        np.where((fico.score.ge(grade_fico[0])), 'CC',
        'C'))))))))))))))
    logging.info("get fico score and 9 level succeed")
    return fico

In [35]:
test_sample_10=pd.read_csv('test_sample_10w.csv')
test_sample_10

,company_id,score_avg,apno_cnt,value_sum,granted_invention_cnt_pb_original_2y
0,C780FDE35B2CFE254E38D038D66E44AC,30.718810,23.666667,1.773000e+05,0.000000
1,A5240D14CD6C29AA3FBC47F6F3362455,22.497244,23.000000,5.510000e+04,0.000000
2,6265D3F1696E2BCACD1C8021FFD6BC73,28.352377,19.000000,1.046000e+05,0.000000
3,5584A75FF07B1B88BE9CEAF534BFAA19,29.501472,18.000000,1.287667e+05,3.000000
4,9717679C5F33A63C015811F5AD67F914,28.501999,14.666667,3.576667e+04,0.000000
...,...,...,...,...,...
100419,A3DE8C8789D8A2ED34C3C7D8BB43AF8D,62.875684,121.666667,1.579400e+06,5.333333
100420,1DF5B678911EFBAA50B1E0D726A4D9DB,60.428449,376.333333,3.260100e+06,8.000000
100421,48595C3A28A3B61407AE1B3101C33F93,65.339886,58.000000,5.667333e+05,7.333333
100422,EDC2E752BE8CEEC3373090933D36CEC1,62.250977,102.000000,1.179100e+06,3.666667


In [36]:
rating_v2=pd.read_csv('/Users/caicaiqijun/智慧牙/数据/TFFI/全国数据/模型/评级趋势优化/rating_score.csv')
grade=rating_v2['score_down'][::-1][1::].reset_index(drop=True).to_list()

In [37]:
validate_data=test_sample_10.rename({'score_avg':'score'},axis=1)

In [38]:
validate_rating=grade_fico(validate_data,grade)

[0.547162092, 4.999257362, 20.44603144, 33.70450675, 45.46051196, 56.58295151, 66.67452317, 75.62709131]


In [59]:
validate_rating=pd.merge(validate_rating,level,on='rating_v2')
validate_rating

,company_id,score,rating_v2,level9
0,C780FDE35B2CFE254E38D038D66E44AC,30.718810,B,6
1,A5240D14CD6C29AA3FBC47F6F3362455,22.497244,B,6
2,6265D3F1696E2BCACD1C8021FFD6BC73,28.352377,B,6
3,5584A75FF07B1B88BE9CEAF534BFAA19,29.501472,B,6
4,DEEB287C79A7C102469C75A3B63752A2,31.928428,B,6
...,...,...,...,...
100419,7EC8ABC7341AAE3D83469419316B1AA8,59.266104,A,3
100420,A3DE8C8789D8A2ED34C3C7D8BB43AF8D,62.875684,A,3
100421,1DF5B678911EFBAA50B1E0D726A4D9DB,60.428449,A,3
100422,48595C3A28A3B61407AE1B3101C33F93,65.339886,A,3


# 开发结果

In [49]:
result=pd.read_parquet('alpha0.6_beta=0(1).2_damped_len=24damped=True')
result

,company_id,predict_score,rating_v2
0,8C42C27B5A781A96925368FC11DA56E2,3.617061,CC
1,81324ECFE05066A68AF69CCF1EE2060F,31.196643,B
2,3D89FE71EEB374940EB73430724C637A,11.590732,CCC
3,1D5917011A4F935F4641E8C98AA13702,10.650894,CCC
4,15D77DBA1ABBA8F0238B193ECCF15597,4.159753,CC
...,...,...,...
100405,D2E34DE1C4109D3812EDCDC597AAC37A,9.389684,CCC
100406,26AAC89192E95BD01FCDEF68ED5AD8D0,1.638114,CC
100407,B1E7D490982381AF4BFD4F8E7899C0AD,4.740658,CC
100408,178C2BD9661C50E74DAB28D2797DD5C8,8.022736,CCC


In [60]:
result=pd.merge(result,level,on='rating_v2')
result

,company_id,predict_score,rating_v2,level9
0,8C42C27B5A781A96925368FC11DA56E2,3.617061,CC,8
1,15D77DBA1ABBA8F0238B193ECCF15597,4.159753,CC,8
2,F4E1C285D40AD9481123FCA316FC4A4F,2.372470,CC,8
3,A504896F4742A65D51BC785691D9847C,2.028293,CC,8
4,030C075EEB150181E47C1701A879074A,2.261706,CC,8
...,...,...,...,...
100397,142BDD0C64F4A5EFF70AB517F826E30C,58.410366,A,3
100398,A3DE8C8789D8A2ED34C3C7D8BB43AF8D,63.075977,A,3
100399,00FCB9092015A0642940DF8E82F1B8DD,64.979939,A,3
100400,344CDE97AEF461ABE24DFCD5FB154149,56.983772,A,3


# 对比

In [61]:
compare=pd.merge(validate_rating,result,on='company_id',suffixes=['_test','_dev'])
compare

,company_id,score,rating_v2_test,level9_test,predict_score,rating_v2_dev,level9_dev
0,C780FDE35B2CFE254E38D038D66E44AC,30.718810,B,6,31.428057,B,6
1,A5240D14CD6C29AA3FBC47F6F3362455,22.497244,B,6,22.807227,B,6
2,6265D3F1696E2BCACD1C8021FFD6BC73,28.352377,B,6,30.494021,B,6
3,5584A75FF07B1B88BE9CEAF534BFAA19,29.501472,B,6,34.082422,BB,5
4,DEEB287C79A7C102469C75A3B63752A2,31.928428,B,6,31.776526,B,6
...,...,...,...,...,...,...,...
100397,7EC8ABC7341AAE3D83469419316B1AA8,59.266104,A,3,61.245895,A,3
100398,A3DE8C8789D8A2ED34C3C7D8BB43AF8D,62.875684,A,3,63.075977,A,3
100399,1DF5B678911EFBAA50B1E0D726A4D9DB,60.428449,A,3,61.957969,A,3
100400,48595C3A28A3B61407AE1B3101C33F93,65.339886,A,3,68.788666,A,3


In [57]:
perform = pd.DataFrame(classification_report(compare['rating_v2_test'].tolist(), compare['rating_v2_dev'].tolist(), output_dict = 'dict')).transpose()
perform.sort_values('f1-score', ascending = False)

,precision,recall,f1-score,support
AAA,1.000000,1.000000,1.000000,28.000000
CCC,0.884615,0.908019,0.896164,41389.000000
C,0.882687,0.884653,0.883669,15267.000000
macro avg,0.873191,0.877394,0.874865,100402.000000
accuracy,0.869614,0.869614,0.869614,0.869614
AA,0.860927,0.878378,0.869565,148.000000
weighted avg,0.869880,0.869614,0.869033,100402.000000
BB,0.854221,0.869347,0.861717,4179.000000
B,0.821039,0.897621,0.857624,11643.000000
A,0.854369,0.838095,0.846154,525.000000


In [62]:
compare['level_gap']=abs(compare['level9_test']-compare['level9_dev'])
compare

,company_id,score,rating_v2_test,level9_test,predict_score,rating_v2_dev,level9_dev,level_gap
0,C780FDE35B2CFE254E38D038D66E44AC,30.718810,B,6,31.428057,B,6,0
1,A5240D14CD6C29AA3FBC47F6F3362455,22.497244,B,6,22.807227,B,6,0
2,6265D3F1696E2BCACD1C8021FFD6BC73,28.352377,B,6,30.494021,B,6,0
3,5584A75FF07B1B88BE9CEAF534BFAA19,29.501472,B,6,34.082422,BB,5,1
4,DEEB287C79A7C102469C75A3B63752A2,31.928428,B,6,31.776526,B,6,0
...,...,...,...,...,...,...,...,...
100397,7EC8ABC7341AAE3D83469419316B1AA8,59.266104,A,3,61.245895,A,3,0
100398,A3DE8C8789D8A2ED34C3C7D8BB43AF8D,62.875684,A,3,63.075977,A,3,0
100399,1DF5B678911EFBAA50B1E0D726A4D9DB,60.428449,A,3,61.957969,A,3,0
100400,48595C3A28A3B61407AE1B3101C33F93,65.339886,A,3,68.788666,A,3,0


In [65]:
compare['level_gap'].value_counts()

0    87311
1    12554
2      511
3       19
4        7
Name: level_gap, dtype: int64

In [64]:
compare.sort_values('level_gap',ascending=False)

,company_id,score,rating_v2_test,level9_test,predict_score,rating_v2_dev,level9_dev,level_gap
97991,030E1A8E1671A8C90D685DBDD9817116,39.494203,BB,5,0.000000,C,9,4
98838,EDC8253600F33CA5503C8EE3814AE737,33.830011,BB,5,0.000000,C,9,4
97882,B44EA85E586C729DB3B725CABA1583AA,40.893664,BB,5,0.000000,C,9,4
98643,71FC05B0AD5655477594DBA18D29B423,37.282114,BB,5,0.000000,C,9,4
96342,336919EC2AA838EBC29B2E16A4ED6856,38.660978,BB,5,0.000000,C,9,4
...,...,...,...,...,...,...,...,...
35167,56C25A4E685B2AD4DCF84234C7410416,6.259310,CCC,7,6.403515,CCC,7,0
35166,3447B85E4E63E1FBFBF34A629532397D,10.454238,CCC,7,8.781422,CCC,7,0
35165,1859CE3DDEB623056DA823B4BEF7815E,9.363979,CCC,7,10.714903,CCC,7,0
35164,68AAB6ED329D4A725FE16A93E99CAD76,18.701232,CCC,7,20.840685,CCC,7,0


In [71]:
compare.query('level_gap==1')

,company_id,score,rating_v2_test,level9_test,predict_score,rating_v2_dev,level9_dev,level_gap
3,5584A75FF07B1B88BE9CEAF534BFAA19,29.501472,B,6,34.082422,BB,5,1
5,505A01504CAECC25E19283BF2C81BD10,20.545849,B,6,19.043123,CCC,7,1
16,4CD460F28CB78EC014E2646214BEF100,21.005643,B,6,19.849308,CCC,7,1
26,9802DB770532CBDAABCC10794E97C89A,24.894050,B,6,21.823271,CCC,7,1
27,E87BCEE105442F97F82B22846E9AE18D,33.125511,B,6,34.376482,BB,5,1
...,...,...,...,...,...,...,...,...
100366,FA89FE0ED83542CD5082B8039B6D75CE,57.346260,A,3,54.460120,BBB,4,1
100367,55C489852E870429D6515276CF29A6E0,56.769073,A,3,52.020504,BBB,4,1
100379,221907B7357D5C5127316ED9DBBDFA0F,57.245798,A,3,54.314727,BBB,4,1
100386,530E8A4F0ADD4D3E53425C1028B5FD1B,62.736729,A,3,67.172445,AA,2,1
